In [1]:
import pandas as pd
import numpy as np

In [2]:
dfp = pd.read_json('reddit_pat.json', lines=True)
dfk = pd.read_json('reddit_kv.json', lines=True)

In [3]:
print(dfp.shape)
display(dfp.head())
print(dfk.shape)
display(dfk.head())

(2450, 5)


,age,num_comments,score,text,timestamp
0,6 hours ago,1067,16886,Pakistan will release Indian Pilot Abhinandan ...,2019-02-28 11:16:12
1,10 hours ago,5338,43583,Trump-Kim talks end 'without agreement',2019-02-28 07:01:12
2,7 hours ago,1099,4036,UN Council: Israel Intentionally Shot Children...,2019-02-28 10:47:58
3,1 hour ago,89,592,Israel's Attorney General Avichai Mendelblit a...,2019-02-28 16:05:00
4,2 hours ago,175,663,Cohen connects the dots on what Trump knew abo...,2019-02-28 15:08:29


(2650, 5)


,age,num_comments,score,text,timestamp
0,5 hours ago,1588,21720,China Killing Prisoners To Harvest Organs For ...,2019-06-19 11:49:08
1,4 hours ago,402,2661,Muslim family dragged out of Belgian embassy i...,2019-06-19 12:05:31
2,11 hours ago,3320,46977,Women outperform men after Japan medical schoo...,2019-06-19 05:51:44
3,6 hours ago,202,1474,MH17 crash: Investigators 'to charge four with...,2019-06-19 10:50:51
4,11 hours ago,1336,2665,Iranian official calls on world to unite again...,2019-06-19 05:09:15


In [4]:
reddit_combined = pd.concat([dfk,dfp], ignore_index=True)
reddit_combined.shape

(5100, 5)

In [5]:
reddit_combined.tail()

,age,num_comments,score,text,timestamp
5095,1 hour ago,20,117,"‘Kremlin Used’ NRA to Help Trump in 2016, Says...",2018-05-16 16:55:22
5096,23 hours ago,4356,31969,N. Korea cancels Wednesday's high-level talks ...,2018-05-15 18:31:54
5097,9 hours ago,62,464,Erdoğan ends UK state visit by calling jailed ...,2018-05-16 09:13:12
5098,16 hours ago,126,2007,Oil companies have played a major role in clim...,2018-05-16 01:17:21
5099,13 hours ago,492,1060,Nikki Haley denies Gaza violence is related to...,2018-05-16 04:31:37


Indexes are properly assigned

In [6]:
reddit_combined.drop_duplicates(subset='text', keep='first').shape     # Keeps the first(lowest index)
                                                                    # Which is the later(time) archive
    
    # This also drops all but one of the paywalled items where text = ''(empty but still an object)

(4843, 5)

In [9]:
# To drop the remaining empty lets locate
index_to_drop = reddit_combined[reddit_combined.text==''].index
reddit_combined = reddit_combined.drop(index=index_to_drop, axis=0)

In [10]:
reddit_combined[reddit_combined.text=='']

,age,num_comments,score,text,timestamp


In [11]:
reddit_trimmed=reddit_combined.drop_duplicates(subset='text',keep='first')

In [12]:
reddit_trimmed.to_json('reddit_combined.json')

In [13]:
df = pd.read_json('reddit_combined.json').sort_index()
df.head()

,age,num_comments,score,text,timestamp
0,5 hours ago,1588,21720,China Killing Prisoners To Harvest Organs For ...,2019-06-19 11:49:08
1,4 hours ago,402,2661,Muslim family dragged out of Belgian embassy i...,2019-06-19 12:05:31
2,11 hours ago,3320,46977,Women outperform men after Japan medical schoo...,2019-06-19 05:51:44
3,6 hours ago,202,1474,MH17 crash: Investigators 'to charge four with...,2019-06-19 10:50:51
4,11 hours ago,1336,2665,Iranian official calls on world to unite again...,2019-06-19 05:09:15


In [14]:
df.describe()

,num_comments,score
count,4842.000000,4842.000000
mean,694.874845,8498.009500
std,1409.923698,16761.483698
min,0.000000,17.000000
25%,44.000000,241.000000
50%,165.500000,1099.000000
75%,667.750000,5786.500000
max,27532.000000,142436.000000


In [15]:
df.isna().sum()

age             0
num_comments    0
score           0
text            0
timestamp       0
dtype: int64

In [16]:
df.age.unique()

array(['5 hours ago', '4 hours ago', '11 hours ago', '6 hours ago',
       '2 hours ago', '21 hours ago', '3 hours ago', '16 hours ago',
       '1 day ago', '10 hours ago', '23 hours ago', '13 hours ago',
       '1 hour ago', '14 hours ago', '18 hours ago', '15 hours ago',
       '9 hours ago', '20 hours ago', '17 hours ago', '19 hours ago',
       '12 hours ago', '7 hours ago', '22 hours ago', '8 hours ago',
       '50 minutes ago', '41 minutes ago', '43 minutes ago',
       '45 minutes ago', '47 minutes ago', '22 minutes ago',
       '56 minutes ago', '59 minutes ago', '18 minutes ago',
       '35 minutes ago', '34 minutes ago', '55 minutes ago',
       '29 minutes ago', '33 minutes ago', '54 minutes ago',
       '39 minutes ago', '28 minutes ago', '40 minutes ago',
       '31 minutes ago', '46 minutes ago'], dtype=object)

In [17]:
def clean_age(text):
    """Compute age in minutes"""
    if 'day' in text:
        return 1440
    if 'hour' in text:
        return int(text[:2]) * 60
    else:
        return int(text[:2])

In [18]:
df.loc[:,'weekday_posted'] = df['timestamp'].map(lambda x: x.weekday())
df.loc[:,'hour_posted'] = df['timestamp'].map(lambda x: x.hour)
df['age'] = df['age'].map(clean_age)

In [19]:
df.loc[:,'log_score'] = np.log(df['score'])
df.loc[:,'log_comments'] = np.log(df['num_comments'] + 0.01)

In [20]:
print(df.shape)
df.head()

(4842, 9)


,age,num_comments,score,text,timestamp,weekday_posted,hour_posted,log_score,log_comments
0,300,1588,21720,China Killing Prisoners To Harvest Organs For ...,2019-06-19 11:49:08,2,11,9.985989,7.370237
1,240,402,2661,Muslim family dragged out of Belgian embassy i...,2019-06-19 12:05:31,2,12,7.886457,5.996477
2,660,3320,46977,Women outperform men after Japan medical schoo...,2019-06-19 05:51:44,2,5,10.757413,8.107723
3,360,202,1474,MH17 crash: Investigators 'to charge four with...,2019-06-19 10:50:51,2,10,7.295735,5.308317
4,660,1336,2665,Iranian official calls on world to unite again...,2019-06-19 05:09:15,2,5,7.887959,7.197443


In [21]:
df.to_json('prepped_data.json')

In [22]:
from sklearn.model_selection import train_test_split

In [23]:
# Ideally split so test is newest, we'll worry about that later.
train, test = train_test_split(df, test_size=0.2, random_state=100)

In [95]:
train.reset_index(inplace=True)
test.reset_index(inplace=True)

train.shape, test.shape

((3873, 10), (969, 10))

In [128]:
X_train = train[['text', 'age', 'weekday_posted', 'hour_posted']]
y1_train = train[['log_score']]
y2_train = train[['log_comments']]

X_test = test[['text', 'age', 'weekday_posted', 'hour_posted']]
y1_test = test[['log_score']]
y2_test = test[['log_comments']]

In [129]:
X_train.shape

(3873, 4)

In [130]:
text_body = X_train.text

In [131]:
from sklearn.feature_extraction.text import  TfidfVectorizer

In [132]:
tfidf = TfidfVectorizer()

In [133]:
tfidf.fit(text_body)

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [134]:
tfidf_vectors = pd.DataFrame(tfidf.transform(X_train['text']).toarray())

In [135]:
tfidf_vectors.shape

(3873, 10501)

In [136]:
tfidf_vectors.index

RangeIndex(start=0, stop=3873, step=1)

In [137]:
X_train.index

RangeIndex(start=0, stop=3873, step=1)

In [138]:
display(X_train.head())
display(X_train.tail())
display(tfidf_vectors.head())
tfidf_vectors.tail()

,text,age,weekday_posted,hour_posted
0,Pro-Brexit British ministers are “not living i...,240,6,0
1,Saudi Consul fired and placed under investigat...,1440,2,15
2,A giant inflatable “Tank Man” sculpture has ap...,1200,2,5
3,Russia resists Western calls to free captured ...,600,0,13
4,Secretive Saudi court may soon decide to execu...,900,1,15


,text,age,weekday_posted,hour_posted
3868,The Nobel Prize in medicine has been awarded t...,360,1,1
3869,Trump hints at payback for 'evil' enemies over...,720,1,6
3870,Tiananmen square falls silent as tight securit...,900,1,3
3871,Second largest outbreak of Ebola in Central Af...,1080,5,17
3872,Folic acid to be added to UK flour in effort t...,660,6,21


,0,1,2,3,4,5,6,7,8,9,...,10491,10492,10493,10494,10495,10496,10497,10498,10499,10500
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


,0,1,2,3,4,5,6,7,8,9,...,10491,10492,10493,10494,10495,10496,10497,10498,10499,10500
3868,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3869,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3870,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3871,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3872,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [139]:
X_train = X_train.drop('text', axis=1,)

In [140]:
# Different indexes from pre_split so ignore index
X_train = pd.concat([X_train, tfidf_vectors], axis=1,)

In [141]:
X_train.head()

,age,weekday_posted,hour_posted,0,1,2,3,4,5,6,...,10491,10492,10493,10494,10495,10496,10497,10498,10499,10500
0,240,6,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1440,2,15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1200,2,5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,600,0,13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,900,1,15,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [143]:
X_train = pd.get_dummies(X_train, columns=['weekday_posted', 'hour_posted'])

In [144]:
X_train.head()

,age,0,1,2,3,4,5,6,7,8,...,hour_posted_14,hour_posted_15,hour_posted_16,hour_posted_17,hour_posted_18,hour_posted_19,hour_posted_20,hour_posted_21,hour_posted_22,hour_posted_23
0,240,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,1440,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,0,0,0,0,0
2,1200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,600,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,900,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,1,0,0,0,0,0,0,0,0


In [145]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor

In [146]:
lr_model = LinearRegression()
lr_model.fit(X_train, y1_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [155]:
tfidf_vectors_test = pd.DataFrame(tfidf.transform(X_test['text']).toarray())

In [156]:
X_test = pd.concat([X_test, tfidf_vectors_test], axis=1)

In [157]:
X_test = X_test.drop('text', axis=1)
X_test = pd.get_dummies(X_test, columns=['hour_posted', 'weekday_posted'])

In [158]:
preds = lr_model.predict(X_test)

In [159]:
unlogged_preds = np.exp(preds)

In [160]:
from sklearn.metrics import mean_absolute_error

In [161]:
mean_absolute_error(y1_test, unlogged_preds)

21704.856947722576

In [163]:
lr_model.score(X_test, np.log(y1_test))

-409.7213025616863

THIS IS HILARIOUSLY BAD, DO NOT REPEAT